In [1]:
# These are all the modules I'll be using later. 
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

import pandas as pd
import urllib
import cPickle as pickle
#from PIL import Image
import scipy.io as sio
import scipy.io
import h5py
import random
import tensorflow as tf
from sklearn.cross_validation import train_test_split as ttsplit
import math as math
#import pylab as

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# I used udacity's discussion(https://discussions.udacity.com/t/how-to-deal-with-mat-files/160657/5) for reference.

class DigitStructFile:
  def __init__(self, inf):
    self.inf = h5py.File(inf, 'r')
    self.digitStructName = self.inf['digitStruct']['name']
    self.digitStructBbox = self.inf['digitStruct']['bbox']

# getName returns the 'name' string for for the n(th) digitStruct. 
  def getName(self,n):
    return ''.join([chr(c[0]) for c in self.inf[self.digitStructName[n][0]].value])

# bboxHelper handles the coding difference when there is exactly one bbox or an array of bbox. 
  def bboxHelper(self,attr):
    if (len(attr) > 1):
      attr = [self.inf[attr.value[j].item()].value[0][0] for j in range(len(attr))]
    else:
      attr = [attr.value[0][0]]
    return attr

# getBbox returns a dict of data for the n(th) bbox. 
  def getBbox(self,n):
    bbox = {}
    bb = self.digitStructBbox[n].item()
    bbox['height'] = self.bboxHelper(self.inf[bb]["height"])
    bbox['label'] = self.bboxHelper(self.inf[bb]["label"])
    bbox['left'] = self.bboxHelper(self.inf[bb]["left"])
    bbox['top'] = self.bboxHelper(self.inf[bb]["top"])
    bbox['width'] = self.bboxHelper(self.inf[bb]["width"])
    return bbox

  def getDigitStructure(self,n):
    s = self.getBbox(n)
    s['name']=self.getName(n)
    return s

# getAllDigitStructure returns all the digitStruct from the input file.     
  def getAllDigitStructure(self):
    return [self.getDigitStructure(i) for i in range(len(self.digitStructName))]

# Return a restructured version of the dataset (one structure by boxed digit).
#
#   Return a list of such dicts :
#      'filename' : filename of the samples
#      'boxes' : list of such dicts (one by digit) :
#          'label' : 1 to 9 corresponding digits. 10 for digit '0' in image.
#          'left', 'top' : position of bounding box
#          'width', 'height' : dimension of bounding box
#
# Note: We may turn this to a generator, if memory issues arise.
  def getAllDigitStructure_ByDigit(self):
        pictDat = self.getAllDigitStructure()
        result = []
        structCnt = 1
        for i in range(len(pictDat)):
            item = { 'filename' : pictDat[i]["name"] }
            figures = []
            for j in range(len(pictDat[i]['height'])):
               figure = {}
               figure['height'] = pictDat[i]['height'][j]
               figure['label']  = pictDat[i]['label'][j]
               figure['left']   = pictDat[i]['left'][j]
               figure['top']    = pictDat[i]['top'][j]
               figure['width']  = pictDat[i]['width'][j]
               figures.append(figure)
            structCnt = structCnt + 1
            item['boxes'] = figures
            result.append(item)
        return result

In [3]:
train_folders = 'train'
test_folders = 'test'
extra_folders = 'extra'

In [7]:
f = os.path.join(train_folders,'digitStruct.mat')
dat = DigitStructFile(f)
train_dataset = dat.getAllDigitStructure_ByDigit()

f2= os.path.join(test_folders,'digitStruct.mat')
dat2 = DigitStructFile(f2)
test_dataset = dat2.getAllDigitStructure_ByDigit()

f3 = os.path.join(extra_folders,'digitStruct.mat')
dat3 = DigitStructFile(f3)
extra_dataset = dat3.getAllDigitStructure_ByDigit()

In [8]:
test_dataset[0]

{'boxes': [{'height': 30.0,
   'label': 5.0,
   'left': 43.0,
   'top': 7.0,
   'width': 19.0}],
 'filename': '1.png'}

In [9]:
def maybe_pickle(dataset, folders, force=False):
  set_filename = folders + '.pickle'
  if os.path.exists(set_filename) and not force:
    print('%s already present - Skipping pickling.' % set_filename)
  else:
    print('Pickling %s.' % set_filename)
    try:
      with open(set_filename, 'wb') as f:
        pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      print('Seve', set_filename, '!')
    except Exception as e:
      print('Unable to save data to', set_filename, ':', e)

In [11]:
maybe_pickle(train_dataset,train_folders)
maybe_pickle(test_dataset,test_folders)
maybe_pickle(extra_dataset,extra_folders)

Pickling train.pickle.
Seve train.pickle !
Pickling test.pickle.
Seve test.pickle !
Pickling extra.pickle.
Seve extra.pickle !
